In [7]:
pip install selenium webdriver-manager


   ---------------------------------------- 0.0/9.5 MB ? eta -:--:--
    --------------------------------------- 0.2/9.5 MB 6.7 MB/s eta 0:00:02
   - -------------------------------------- 0.3/9.5 MB 4.1 MB/s eta 0:00:03
   -- ------------------------------------- 0.7/9.5 MB 4.9 MB/s eta 0:00:02
   ----- ---------------------------------- 1.2/9.5 MB 6.4 MB/s eta 0:00:02
   ------ --------------------------------- 1.5/9.5 MB 6.3 MB/s eta 0:00:02
   ------- -------------------------------- 1.8/9.5 MB 6.3 MB/s eta 0:00:02
   -------- ------------------------------- 2.1/9.5 MB 6.2 MB/s eta 0:00:02
   --------- ------------------------------ 2.3/9.5 MB 6.2 MB/s eta 0:00:02
   ---------- ----------------------------- 2.6/9.5 MB 6.1 MB/s eta 0:00:02
   ----------- ---------------------------- 2.8/9.5 MB 6.0 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.5 MB 5.9 MB/s eta 0:00:02
   ------------- -------------------------- 3.2/9.5 MB 5.9 MB/s eta 0:00:02
   -------------- 

In [19]:
import time
import csv
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

# Function to initialize Selenium WebDriver
def init_driver():
    options = Options()
    options.add_argument("--headless")  # Run in headless mode
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")  # Prevent detection
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    return driver

# Function to scroll down and load more reviews
def scroll_down(driver):
    last_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(5):  # Scroll multiple times to load reviews
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Wait for reviews to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

# Function to scrape reviews from Shopee
def scrape_reviews(url):
    driver = init_driver()
    driver.get(url)

    try:
        # Wait until the first review element is present
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shopee-product-rating"))
        )
        
        scroll_down(driver)  # Scroll to load more reviews

        reviews = []
        review_elements = driver.find_elements(By.CLASS_NAME, "shopee-product-rating")

        if not review_elements:
            print("No reviews found. The webpage structure may have changed.")
            return []

        for review in review_elements:
            try:
                name = review.find_element(By.CLASS_NAME, "shopee-product-rating__author-name").text
                date = review.find_element(By.CLASS_NAME, "shopee-product-rating__time").text
                content_element = review.find_elements(By.CLASS_NAME, "shopee-product-rating__content")
                content = content_element[0].text if content_element else "No review text"

                reviews.append([name, date, content])
            except Exception as e:
                print(f"Error extracting review: {e}")

    except Exception as e:
        print(f"Error loading reviews: {e}")
        reviews = []

    driver.quit()
    return reviews

# Function to save reviews to CSV
def save_to_csv(reviews, filename="review.csv"):
    try:
        if os.path.exists(filename):
            os.remove(filename)  # Delete existing file to prevent permission issues
        
        with open(filename, mode="w", newline="", encoding="utf-8") as file:
            writer = csv.writer(file)
            writer.writerow(["Reviewer Name", "Review Date", "Review Content"])
            writer.writerows(reviews)
        print(f"Reviews saved successfully to {filename}")
    except PermissionError:
        print("Error: Permission denied. Close the file if it's open and try again.")
    except Exception as e:
        print(f"Unexpected error saving CSV: {e}")

# Main execution
if __name__ == "__main__":
    shopee_url = "https://shopee.com.my/product/94327525/4631783674"
    review_data = scrape_reviews(shopee_url)
    if review_data:
        save_to_csv(review_data)
    else:
        print("No reviews extracted. Check the webpage structure.")


Error loading reviews: Message: 
Stacktrace:
	GetHandleVerifier [0x009C0B43+25139]
	(No symbol) [0x009513F4]
	(No symbol) [0x008304E3]
	(No symbol) [0x008783D7]
	(No symbol) [0x0087872B]
	(No symbol) [0x008C1002]
	(No symbol) [0x0089D014]
	(No symbol) [0x008BE778]
	(No symbol) [0x0089CDC6]
	(No symbol) [0x0086BDE9]
	(No symbol) [0x0086D124]
	GetHandleVerifier [0x00CC4373+3185251]
	GetHandleVerifier [0x00CE291A+3309578]
	GetHandleVerifier [0x00CDCF42+3286578]
	GetHandleVerifier [0x00A57AE0+643536]
	(No symbol) [0x0095A20D]
	(No symbol) [0x009570B8]
	(No symbol) [0x00957257]
	(No symbol) [0x00949E00]
	BaseThreadInitThunk [0x75B55D49+25]
	RtlInitializeExceptionChain [0x7776CE3B+107]
	RtlGetAppContainerNamedObjectPath [0x7776CDC1+561]

No reviews extracted. Check the webpage structure.
